<h1>Making pandas dataframe

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_html('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [3]:
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df_filtered = df.loc[df['Borough'] != 'Not assigned', :]
df_filtered.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [4]:
def merge_func(d):
    merged = pd.DataFrame(columns=d.columns)
    merged = merged.append({'PostalCode': d.iloc[0,0], 
                   'Borough': d.iloc[0,1], 
                   'Neighborhood': ','.join(d['Neighborhood'])}, ignore_index=True)
    return merged
    
df_merged = df_filtered.groupby('PostalCode').apply(merge_func).reset_index(drop=True)

In [5]:
# Processing Neighborhood with 'Not assgined'
df_merged.loc[df_merged['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df_merged['Borough']

In [6]:
df_merged.shape

(103, 3)

<h1>Adding Coordinates

In [7]:
df_coords = pd.read_csv('https://cocl.us/Geospatial_data')
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
df_coords.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

df_merged_coords = pd.merge(df_merged, df_coords)
df_merged_coords

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


<h1>Analysis

In [123]:
import folium # map rendering library
from geopy.geocoders import Nominatim
import requests # library to handle requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [33]:
df_toronto = df_merged_coords[df_merged_coords['Borough'].str.contains('Toronto')]
df_toronto = df_toronto.iloc[:, 1:]
df_toronto.head()

,Borough,Neighborhood,Latitude,Longitude
37,East Toronto,The Beaches,43.676357,-79.293031
41,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,East Toronto,Studio District,43.659526,-79.340923
44,Central Toronto,Lawrence Park,43.728020,-79.388790


In [42]:
address = 'Toronto, CA'

geolocator = Nominatim(timeout=30, user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto are 43.653963, -79.387207.


<h2>Get the neaby venues data

In [46]:
CLIENT_ID = 'DSPQMQDMQVKS1TNMP21YPC4NJQR42OJF1QTVMFAGII4W3S4E' # your Foursquare ID
CLIENT_SECRET = '0UQ2GAKTLAH2ML4VDA0HKUY4UGN25N0WGK54TMYJPUVYLYFN' # your Foursquare Secret
VERSION = '20180323'
LIMIT=10

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [56]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [81]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
...,...,...,...,...,...,...,...
334,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,The Green Wood,43.664728,-79.324117,Restaurant
335,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Chick-n-Joy,43.665181,-79.321403,Fast Food Restaurant
336,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Ashbridges Bay Skatepark,43.662548,-79.315631,Skate Park
337,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,East End Garden Centre & Hardware,43.664555,-79.324598,Garden Center


In [82]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",10,10,10,10,10,10
Berczy Park,10,10,10,10,10,10
"Brockton,Exhibition Place,Parkdale Village",10,10,10,10,10,10
Business Reply Mail Processing Centre 969 Eastern,10,10,10,10,10,10
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",10,10,10,10,10,10
"Cabbagetown,St. James Town",10,10,10,10,10,10
Central Bay Street,10,10,10,10,10,10
"Chinatown,Grange Park,Kensington Market",10,10,10,10,10,10
Christie,10,10,10,10,10,10


In [83]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 116 uniques categories.


<h2>Replace 'Neighborhood' in Venue Categoty with 'Neighbor' to avoid conflict with column name

In [108]:
toronto_venues['Venue Category'][toronto_venues['Venue Category']=='Neighborhood'] = 'Neighbor'
toronto_venues['Venue Category'][toronto_venues['Venue Category']=='Neighbor']

d:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


3      Neighbor
30     Neighbor
163    Neighbor
Name: Venue Category, dtype: object

<h2>Add one hot encoding columns

In [109]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,...,Swim School,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West,Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [110]:
toronto_onehot.shape

(339, 117)

<h2> Get the mean of each neighborhood

In [111]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,...,Swim School,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,...,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.1,0.0,0.0
1,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.1,0.0,0.0
2,"Brockton,Exhibition Place,Parkdale Village",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,...,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.1,0.1,0.1,0.2,0.1,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
5,"Cabbagetown,St. James Town",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
6,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
7,"Chinatown,Grange Park,Kensington Market",0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,...,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
8,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
9,Church and Wellesley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.1,0.00,0.0,0.0,0.1,0.00,0.0,0.0,0.0


In [112]:
toronto_grouped.shape

(38, 117)

<h2>Print top 5 venues

In [113]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    #print(temp.head())
    #input('xxx')
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]            # remove header
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                           venue  freq
0                    Opera House   0.1
1                    Pizza Place   0.1
2  Vegetarian / Vegan Restaurant   0.1
3                   Concert Hall   0.1
4                      Speakeasy   0.1


----Berczy Park----
               venue  freq
0       Concert Hall   0.1
1  French Restaurant   0.1
2               Park   0.1
3       Cocktail Bar   0.1
4     Farmers Market   0.1


----Brockton,Exhibition Place,Parkdale Village----
                    venue  freq
0             Coffee Shop   0.2
1  Furniture / Home Store   0.1
2                     Gym   0.1
3               Pet Store   0.1
4                    Café   0.1


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0            Restaurant   0.1
1  Fast Food Restaurant   0.1
2           Pizza Place   0.1
3               Brewery   0.1
4            Skate Park   0.1


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King

<h2>Get most 10 common venues

In [115]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [142]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        #print(ind, indicators[ind])
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Opera House,Pizza Place,Speakeasy,Hotel,Steakhouse,Plaza,Café,Asian Restaurant,Concert Hall,Vegetarian / Vegan Restaurant
1,Berczy Park,Farmers Market,Steakhouse,Concert Hall,Cocktail Bar,Museum,Liquor Store,Breakfast Spot,Park,Vegetarian / Vegan Restaurant,French Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Pet Store,Bar,Caribbean Restaurant,Café,Italian Restaurant,Breakfast Spot,Furniture / Home Store,Gym,French Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Farmers Market,Restaurant,Skate Park,Comic Shop,Fast Food Restaurant,Burrito Place,Auto Workshop,Pizza Place,Garden Center,Brewery
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport,Harbor / Marina,Boutique,Bar,Coffee Shop,Airport Terminal,Airport Gate,Airport Food Court,Dance Studio


In [143]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 4, 2, 4, 0, 4, 0, 0, 4])

In [144]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#print(neighborhoods_venues_sorted.head())
toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,The Beaches,43.676357,-79.293031,2,Trail,Neighbor,Pub,Health Food Store,Yoga Studio,Farmers Market,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant
41,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,2,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Cosmetics Shop,Pub,Brewery,Fast Food Restaurant,Creperie,Cuban Restaurant,Dance Studio
42,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,2,Brewery,Park,Pet Store,Gym,Liquor Store,Italian Restaurant,Fish & Chips Shop,Burger Joint,Sushi Restaurant,Ice Cream Shop
43,East Toronto,Studio District,43.659526,-79.340923,4,Coffee Shop,Bakery,Ice Cream Shop,Fish Market,Neighbor,Café,Sandwich Place,Bookstore,Gay Bar,Auto Workshop
44,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Bus Line,Swim School,Dim Sum Restaurant,Park,Yoga Studio,Fast Food Restaurant,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant


<h2>Categorize venues and create map

In [145]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2>Get the each cluster's data and label with appropriate name.

Cluster 1 - Cafe Street

In [146]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,"Cabbagetown,St. James Town",Café,General Entertainment,Bakery,Diner,Restaurant,Jewelry Store,Italian Restaurant,Indian Restaurant,Japanese Restaurant,Food & Drink Shop
54,"Ryerson,Garden District",Café,Comic Shop,Tea Room,Burrito Place,Ramen Restaurant,Plaza,Pizza Place,Theater,Clothing Store,Cuban Restaurant
61,"Commerce Court,Victoria Hotel",Café,Coffee Shop,Restaurant,Gym,Gym / Fitness Center,Museum,Pub,Gastropub,American Restaurant,Asian Restaurant
65,"The Annex,North Midtown,Yorkville",Café,Coffee Shop,American Restaurant,Indian Restaurant,Pub,BBQ Joint,Burger Joint,Park,Vegetarian / Vegan Restaurant,Dance Studio
67,"Chinatown,Grange Park,Kensington Market",Café,Organic Grocery,Bakery,Mexican Restaurant,Coffee Shop,Cocktail Bar,Arts & Crafts Store,Bar,Eastern European Restaurant,Dog Run
70,"First Canadian Place,Underground city",Café,Gastropub,American Restaurant,Restaurant,Steakhouse,Pizza Place,Gym / Fitness Center,Gym,Coffee Shop,Creperie
75,Christie,Café,Grocery Store,Coffee Shop,Candy Store,Restaurant,Italian Restaurant,Diner,Farmers Market,Cosmetics Shop,Creperie


Cluster 2 - Park Area

In [147]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,"Moore Park,Summerhill East",Park,Trail,Tennis Court,Playground,Comic Shop,Concert Hall,Cosmetics Shop,College Gym,Farmers Market,Creperie
50,Rosedale,Park,Trail,Playground,Yoga Studio,Farmers Market,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant


Cluster 3 - Dessert

In [148]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,The Beaches,Trail,Neighbor,Pub,Health Food Store,Yoga Studio,Farmers Market,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant
41,"The Danforth West,Riverdale",Greek Restaurant,Ice Cream Shop,Italian Restaurant,Cosmetics Shop,Pub,Brewery,Fast Food Restaurant,Creperie,Cuban Restaurant,Dance Studio
42,"The Beaches West,India Bazaar",Brewery,Park,Pet Store,Gym,Liquor Store,Italian Restaurant,Fish & Chips Shop,Burger Joint,Sushi Restaurant,Ice Cream Shop
44,Lawrence Park,Bus Line,Swim School,Dim Sum Restaurant,Park,Yoga Studio,Fast Food Restaurant,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant
45,Davisville North,Breakfast Spot,Hotel,Clothing Store,Sandwich Place,Food & Drink Shop,Park,Gym,Furniture / Home Store,Eastern European Restaurant,Concert Hall
47,Davisville,Dessert Shop,Coffee Shop,Seafood Restaurant,Italian Restaurant,Indian Restaurant,Sushi Restaurant,Café,Pizza Place,Park,Cuban Restaurant
56,Berczy Park,Farmers Market,Steakhouse,Concert Hall,Cocktail Bar,Museum,Liquor Store,Breakfast Spot,Park,Vegetarian / Vegan Restaurant,French Restaurant
58,"Adelaide,King,Richmond",Opera House,Pizza Place,Speakeasy,Hotel,Steakhouse,Plaza,Café,Asian Restaurant,Concert Hall,Vegetarian / Vegan Restaurant
59,"Harbourfront East,Toronto Islands,Union Station",Park,Supermarket,Neighbor,Performing Arts Venue,Lake,Salad Place,Dessert Shop,Sporting Goods Shop,Hotel,New American Restaurant
64,"Forest Hill North,Forest Hill West",Jewelry Store,Trail,Sushi Restaurant,Mexican Restaurant,Yoga Studio,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio


Cluster 4 - Health and Beauty

In [149]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Roselawn,Health & Beauty Service,Garden,Yoga Studio,Fast Food Restaurant,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Dessert Shop


Cluster 5 - Complex

In [150]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Studio District,Coffee Shop,Bakery,Ice Cream Shop,Fish Market,Neighbor,Café,Sandwich Place,Bookstore,Gay Bar,Auto Workshop
46,North Toronto West,Yoga Studio,Sporting Goods Shop,Chinese Restaurant,Dessert Shop,Mexican Restaurant,Restaurant,Salon / Barbershop,Diner,Spa,Coffee Shop
49,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",Coffee Shop,Pub,Sushi Restaurant,Restaurant,American Restaurant,Sports Bar,Liquor Store,Supermarket,Diner,Dim Sum Restaurant
52,Church and Wellesley,Gastropub,Tea Room,Salon / Barbershop,Bubble Tea Shop,Restaurant,Ramen Restaurant,Breakfast Spot,Mexican Restaurant,Dance Studio,Theme Restaurant
53,"Harbourfront,Regent Park",Breakfast Spot,Coffee Shop,Park,Spa,Gym / Fitness Center,Restaurant,Pub,Historic Site,Bakery,Dim Sum Restaurant
55,St. James Town,Coffee Shop,Japanese Restaurant,BBQ Joint,Italian Restaurant,Restaurant,Food Truck,Creperie,Gym,Gastropub,Cuban Restaurant
57,Central Bay Street,Coffee Shop,Sushi Restaurant,Modern European Restaurant,Ramen Restaurant,Italian Restaurant,Gastropub,Spa,Food Truck,Comic Shop,Concert Hall
60,"Design Exchange,Toronto Dominion Centre",Coffee Shop,Beer Bar,Gym,Gym / Fitness Center,Hotel,Pub,Café,Restaurant,Gastropub,Auto Workshop
66,"Harbord,University of Toronto",Italian Restaurant,Bakery,College Gym,Japanese Restaurant,Dessert Shop,French Restaurant,Restaurant,Bookstore,Beer Bar,Bar
68,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport,Harbor / Marina,Boutique,Bar,Coffee Shop,Airport Terminal,Airport Gate,Airport Food Court,Dance Studio
